In [1]:
import logging
logging.basicConfig(level=logging.INFO)

from pystatis import Table

In [4]:
t = Table(name="13231-0001")

In [5]:
t.get_data()

INFO:pystatis.cache:Data was successfully cached under /Users/miay/.pystatis/data/13231-0001/0cf7b54374d707c28edf/20240220.csv.
INFO:pystatis.http_helper:Code 0: erfolgreich


In [6]:
t.name

'13231-0001'

In [17]:
t.raw_data

'Statistik_Code;Statistik_Label;Zeit_Code;Zeit_Label;Zeit;1_Merkmal_Code;1_Merkmal_Label;1_Auspraegung_Code;1_Auspraegung_Label;BEVSTD__Bevoelkerungsstand_Einwohner__Anzahl;WAS005__Einw._m._Anschl._an_d._oeffentl._Wasserversorgung__Anzahl;WAS007__Wasserabgabe_an_Letztverbraucher__1000_cbm;WAS007__Wasserabgabe_an_Letztverbraucher__MeasureUnitNotFound!;WAS008__Wasserabgabe_je_Einwohner_und_Tag_(in_Liter)__MeasureUnitNotFound!;WAS006__Hausbrunnen_oder_Quellen_mit_Trinkwassergewinnung__Anzahl\nKREISE;Kreise und kreisfreie Städte;01001;Flensburg, kreisfreie Stadt;;;;;;;;;;;\nKREISE;Kreise und kreisfreie Städte;01002;Kiel, kreisfreie Stadt;;;;;;;;;;;\nKREISE;Kreise und kreisfreie Städte;01003;Lübeck, kreisfreie Stadt, Hansestadt;;;;;;;;;;;\nKREISE;Kreise und kreisfreie Städte;01004;Neumünster, kreisfreie Stadt;;;;;;;;;;;\nKREISE;Kreise und kreisfreie Städte;01051;Dithmarschen, Kreis;;;;;;;;;;;\nKREISE;Kreise und kreisfreie Städte;01053;Herzogtum Lauenburg, Kreis;;;;;;;;;;;\nKREISE;Kreise und

In [7]:
t.data

,Jahr,Deutschland insgesamt,Monate,Original- und bereinigte Daten,Erwerbspersonen,Erwerbstaetige,Erwerbslose,Erwerbslosenquote
0,2022,Deutschland,Januar,Originalwerte,"43,11","41,68","1,43","3,3"
1,2022,Deutschland,Januar,X13 JDemetra+ Trend,"43,27","41,91","1,36","3,1"
2,2022,Deutschland,Januar,BV4.1 Trend,"43,38","42,02","1,36","3,1"
3,2022,Deutschland,Februar,Originalwerte,"43,17","41,77","1,40","3,2"
4,2022,Deutschland,Februar,X13 JDemetra+ Trend,"43,38","42,04","1,34","3,1"
...,...,...,...,...,...,...,...,...
67,2023,Deutschland,November,X13 JDemetra+ Trend,"44,38","43,01","1,37","3,1"
68,2023,Deutschland,November,BV4.1 Trend,"44,41","43,06","1,35","3,0"
69,2023,Deutschland,Dezember,Originalwerte,"44,51","43,21","1,30","2,9"
70,2023,Deutschland,Dezember,X13 JDemetra+ Trend,"44,46","43,09","1,37","3,1"


In [8]:
from pprint import pprint

pprint(t.metadata)

{'Copyright': '© Statistisches Bundesamt (Destatis), 2024',
 'Ident': {'Method': 'table', 'Service': 'metadata'},
 'Object': {'Code': '13231-0001',
            'Content': 'Erwerbslose, Erwerbstätige, Erwerbspersonen,\n'
                       'Erwerbslosenquote: Deutschland, Monate, Original- und\n'
                       'bereinigte Daten',
            'Structure': {'Columns': [{'Code': 'WERT06',
                                       'Content': 'Original- und bereinigte '
                                                  'Daten',
                                       'Selected': None,
                                       'Structure': [{'Code': 'ERW001',
                                                      'Content': 'Erwerbspersonen',
                                                      'Selected': None,
                                                      'Structure': None,
                                                      'Type': 'Merkmal',
                               